# script per l'importazione dati da file IMPRESE.xlsx 
## fonte infocamere - servizio parix elaborato da INSIEL

i dati sono contenuti in 2 fogli di lavoro
vengono convertiti in 2 file csv 

per approfondire: trattare i csv è più veloce rispetto a xls e consente condivisione nel data repository) cfr: https://stackoverflow.com/questions/50695778/how-to-increase-process-speed-using-read-excel-in-pandas

* creazione id_sede (corrisponde a CF, ma 
                        è più efficiente 
                        non è "dato personale")

* creazione id_impresa (un'impresa ha più sedi identificate da un numero 0 = SEDE , 1 = UL1...)

directory\file di input e output:
        ....

operazioni preliminari:
    1- spostare il file IMPRESE.xlsx nella directory di lavoro

operazioni eseguite da questo script

 



## setup



In [1]:
# Setup
import sys
import os
from pathlib import Path
import datetime
import pandas as pd
pd.options.display.max_columns = None

In [2]:
current_path = Path(os.getcwd())
data_subdir = "dati_imprese"
data_path = current_path.parent / data_subdir
data_dir = str(data_path)
listafile = os.listdir(data_path)
listafile = list(filter(lambda x: 'IMPRESE' in x, listafile)) #filtra solo i file che contengono IMPRESE
assert len(listafile) == 1
assert listafile[0] == 'IMPRESE.xlsx'

excel_file  = data_dir + '\\IMPRESE.xlsx'

print(f'Carico i dati presenti nel file{excel_file}')



Carico i dati presenti nel filec:\Users\morea\OneDrive - Area Science Park\Documents\_DataScience\notebook_imprese_insiel\dati_imprese\IMPRESE.xlsx


In [3]:
#opzione 1: uso il metodo ExcelFile per generare la classe xl
xl = pd.ExcelFile(  excel_file, engine="openpyxl")
xl_sheets = xl.sheet_names  # see all sheet names
assert xl_sheets == ['FRIULI Anagrafica', 'FRIULI codice attività']


In [4]:
#FRIULI Anagrafica
#NOTA: le intestazioni di colonna nel file excel sono molto scadenti:
#  sono distribuite in tre righe e nessuna delle righe è completa. 
# ci sono alcuni nomi di colonna che contengono l'apostrofo >>> attenzione alle stringhe
# correzioni da fare in excel: le colonne Q, AO-AU, attualmente vuote, devono essere riempite con i valori della 2a riga. 

# usiamo la 3a riga (header = 2) 

#NB la lista di colonne ha un ordine che viene mantenuto in fase di lettura
cols_source = [ 'DENOMINAZIONE',
            'c fiscale', 'PARTITA IVA', 'SEZ-REG-IMP', 'N-REG-IMP', 'rea','N-ALBO-AA', 
            'SIGLA PROVINCIA',  'UL-SEDE',
            'NG - Natura Giuridica', 'NG', 'tipo impresa', 
            'DT-COST', 'DT-ISCR-RI', 'DT-ISCR-RD', 'DT-ISCR-AA', 'DT-FINE-AA', 
            'DT-INI-AT', 'DT-APER-UL', 'DT-LIQUID',                        
            'Cancellazione',  'dt cessazione attività', 'Fallimento',
            'AA-ADD', 'IND', 'DIP', 
             'CAPITALE', 'VALUTA-CAPITALE',
             "ATTIVITA'", 
            'stato impresa/ul', 'Tipo Localizzazione',
            'tipo sede/ul1', 'tipo sede/ul2', 'tipo sede/ul3', 'tipo sede/ul4', 'tipo sede/ul5',
            "Presenza di sedi secondarie all'estero",
            'Impresa estera con unità locale in Friuli Venezia Giulia',
            'Indicazione delle imprese che sono PMI',
             'Indicazione delle imprese che sono start up', 'Impr Femminile',
            'Impr Giovane', 'Impr Straniera',
            'INDIRIZZO', 'STRAD', 'CAP', 'COMUNE', 'FRAZIONE', 'ALTRE-INDICAZIONI',  'pec','TELEFONO'
             ]

 #per leggere i tipi di dati usare converters  
 #converters = {col: str for col in column_list} 
 #per le date converters= {'DT-ISCR-RI': pd.to_datetime}
 # df = pd.read_excel('some_excelfile.xls', converters=converters)
 #oppure dtype={'column_name': datetime.datetime}

# da fare: STRIP caratteri non leggibili e falsi separatori (ce ne sono tanti)

In [5]:
# carica i dati nel datafram df_anagrafica
#NB la lista alla fine dell'istruzione df_anagrafica =... impone l'ordine delle colonne

df_anagrafica = xl.parse(   'FRIULI Anagrafica', 
                            header = 2, 
                            dtype=str,
                            usecols = cols_source)[cols_source]



In [6]:
df_anagrafica

,DENOMINAZIONE,c fiscale,PARTITA IVA,SEZ-REG-IMP,N-REG-IMP,rea,N-ALBO-AA,SIGLA PROVINCIA,UL-SEDE,NG - Natura Giuridica,NG,tipo impresa,DT-COST,DT-ISCR-RI,DT-ISCR-RD,DT-ISCR-AA,DT-FINE-AA,DT-INI-AT,DT-APER-UL,DT-LIQUID,Cancellazione,dt cessazione attività,Fallimento,AA-ADD,IND,DIP,CAPITALE,VALUTA-CAPITALE,ATTIVITA',stato impresa/ul,Tipo Localizzazione,tipo sede/ul1,tipo sede/ul2,tipo sede/ul3,tipo sede/ul4,tipo sede/ul5,Presenza di sedi secondarie all'estero,Impresa estera con unità locale in Friuli Venezia Giulia,Indicazione delle imprese che sono PMI,Indicazione delle imprese che sono start up,Impr Femminile,Impr Giovane,Impr Straniera,INDIRIZZO,STRAD,CAP,COMUNE,FRAZIONE,ALTRE-INDICAZIONI,pec,TELEFONO
0,PELLIZZARI SILVIO DI SEVERINO PELLIZZARI E C. ...,00000470310,00000470310,O,(GO007-1352),37843,NaN,GO,SEDE,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4974-08-26 00:00:00,4996-02-19 00:00:00,4975-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,COMMERCIO AL MINUTO TAB. MERC. I - II (SOLTANT...,INATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA PESCHERIA 4,NaN,34071,CORMONS - GO,NaN,NaN,PELLIZZARI@PEC.CGN.IT,0481/60323
1,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,SEDE,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,4969-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA CORTI 2,NaN,34123,TRIESTE - TS,NaN,NaN,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
2,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,UL-2,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5007-08-01 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,FERNETTI 5,NaN,34016,MONRUPINO - TS,NaN,"AUTOPORTO, MODULO 47, PIANO 1",CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
3,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,UL-4,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5015-10-15 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,"PUNTO FRANCO NUOVO ""EX CULP"" SN",NaN,34123,TRIESTE - TS,NaN,NaN,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
4,MOBILI BATTISTON & LOT - S.N.C. DI BATTISTON G...,00002130938,00002130938,O,(PN033-2369),18374,NaN,PN,UL-1,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4973-10-09 00:00:00,4996-02-19 00:00:00,4973-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5008-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INATTIVA,UL - UNITÀ LOCALE,DEP - DEPOSITO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA MEDUNA 33,02330,33170,PORDENONE - PN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127169,ZOZZOLI ANTONIO,ZZZNTN78H14L195O,02564170302,A-P,(UD-2009-7369),270774,UD-87280,UD,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,NaN,5009-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,"ATTIVITA':\nCOSTRUZIONE, RISTRUTTURAZIONE DI E...",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA AONESI 42,NaN,33027,PAULARO - UD,NaN,NaN,ANTONIO.ZOZZOLI@PEC.IT,328/8878256
127170,ESTETICA SAGITTARIO DI ZAZZERON ROBERTA,ZZZRRT70L67L424I,01139490328,A-P,(TS-2008-2532),127748,TS-25173,TS,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,500

In [7]:
df_anagrafica.reset_index(inplace=True)
df_anagrafica['index'] = df_anagrafica['index'] + 1
#df_anagrafica['index'] = df_anagrafica['index'].apply(str)
df_anagrafica.rename(columns={"index": "id_localiz"}, inplace=True) 
#df_anagrafica['n_sede'] = df_anagrafica['index'] + 1

df_anagrafica

,id_localiz,DENOMINAZIONE,c fiscale,PARTITA IVA,SEZ-REG-IMP,N-REG-IMP,rea,N-ALBO-AA,SIGLA PROVINCIA,UL-SEDE,NG - Natura Giuridica,NG,tipo impresa,DT-COST,DT-ISCR-RI,DT-ISCR-RD,DT-ISCR-AA,DT-FINE-AA,DT-INI-AT,DT-APER-UL,DT-LIQUID,Cancellazione,dt cessazione attività,Fallimento,AA-ADD,IND,DIP,CAPITALE,VALUTA-CAPITALE,ATTIVITA',stato impresa/ul,Tipo Localizzazione,tipo sede/ul1,tipo sede/ul2,tipo sede/ul3,tipo sede/ul4,tipo sede/ul5,Presenza di sedi secondarie all'estero,Impresa estera con unità locale in Friuli Venezia Giulia,Indicazione delle imprese che sono PMI,Indicazione delle imprese che sono start up,Impr Femminile,Impr Giovane,Impr Straniera,INDIRIZZO,STRAD,CAP,COMUNE,FRAZIONE,ALTRE-INDICAZIONI,pec,TELEFONO
0,1,PELLIZZARI SILVIO DI SEVERINO PELLIZZARI E C. ...,00000470310,00000470310,O,(GO007-1352),37843,NaN,GO,SEDE,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4974-08-26 00:00:00,4996-02-19 00:00:00,4975-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,COMMERCIO AL MINUTO TAB. MERC. I - II (SOLTANT...,INATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA PESCHERIA 4,NaN,34071,CORMONS - GO,NaN,NaN,PELLIZZARI@PEC.CGN.IT,0481/60323
1,2,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,SEDE,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,4969-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA CORTI 2,NaN,34123,TRIESTE - TS,NaN,NaN,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
2,3,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,UL-2,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5007-08-01 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,FERNETTI 5,NaN,34016,MONRUPINO - TS,NaN,"AUTOPORTO, MODULO 47, PIANO 1",CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
3,4,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,(TS006-7084),65026,NaN,TS,UL-4,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5015-10-15 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,"PUNTO FRANCO NUOVO ""EX CULP"" SN",NaN,34123,TRIESTE - TS,NaN,NaN,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,040/3220798
4,5,MOBILI BATTISTON & LOT - S.N.C. DI BATTISTON G...,00002130938,00002130938,O,(PN033-2369),18374,NaN,PN,UL-1,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4973-10-09 00:00:00,4996-02-19 00:00:00,4973-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5008-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INATTIVA,UL - UNITÀ LOCALE,DEP - DEPOSITO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA MEDUNA 33,02330,33170,PORDENONE - PN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127169,127170,ZOZZOLI ANTONIO,ZZZNTN78H14L195O,02564170302,A-P,(UD-2009-7369),270774,UD-87280,UD,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,NaN,5009-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,"ATTIVITA':\nCOSTRUZIONE, RISTRUTTURAZIONE DI E...",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,VIA AONESI 42,NaN,33027,PAULARO - UD,NaN,NaN,ANTONIO.ZOZZOLI@PEC.IT,328/8878256
127170,127171,ESTETICA SAGITTARIO DI ZAZZERON ROBERTA,ZZZRRT70L67L424I,01139490328,A-P,(TS-2008-2532),127748,TS-25173,TS,SEDE,DI,DI - IMPRESA 

In [8]:
df_anagrafica['n_sede'] = df_anagrafica['UL-SEDE'].str[3:]
df_anagrafica.loc[ df_anagrafica['n_sede'] == 'E', 'n_sede'] = 0
#df_anagrafica.loc[ df_anagrafica['UL-SEDE'] != 'SEDE', 'n_sede'] = df_anagrafica['UL-SEDE'].str[3:].astype(int)
df_anagrafica['n_sede'].unique() 

array([0, '2', '4', '1', '3', '11', '7', '8', '5', '6', '12', '18', '9',
       '10', '20', '19', '15', '21', '100', '101', '102', '103', '104',
       '105', '106', '107', '108', '109', '110', '111', '112', '113',
       '16', '27', '28', '29', '32', '35', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '59', '65', '67', '69', '70', '71', '72', '73',
       '74', '75', '76', '78', '79', '80', '81', '82', '83', '84', '85',
       '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '98',
       '99', '58', '60', '61', '13', '14', '17', '22', '23', '26', '30',
       '31', '33', '36', '38', '62', '63', '66', '24', '25', '37', '34',
       '77', '86', '68', '64', '124', '120', '121', '115', '114', '117',
       '123', '118', '119', '116', '122', '97', '217', '218', '219',
       '220', '221', '222', '229', '223', '224', '225', '226', '227',
       '232', '125', '127', '128', '129', '130', '132', '133',

In [9]:
# visualizza il dataframe e alcune descrizioni 
dim = df_anagrafica.shape #numero di righe e colonne del dataset
print(f'dimensioni del dataset (righe,colonne) = {dim}\n')
print(df_anagrafica.info())


dimensioni del dataset (righe,colonne) = (127174, 53)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127174 entries, 0 to 127173
Data columns (total 53 columns):
 #   Column                                                    Non-Null Count   Dtype 
---  ------                                                    --------------   ----- 
 0   id_localiz                                                127174 non-null  int64 
 1   DENOMINAZIONE                                             127174 non-null  object
 2   c fiscale                                                 127174 non-null  object
 3   PARTITA IVA                                               126334 non-null  object
 4   SEZ-REG-IMP                                               119528 non-null  object
 5   N-REG-IMP                                                 127174 non-null  object
 6   rea                                                       127174 non-null  object
 7   N-ALBO-AA                                     

In [10]:
cols = df_anagrafica.columns
if len(cols) != len(cols_source):
    print('numero di colonne non corrispondente')
    check_list = [item in cols  for item in cols_source]
    if False in check_list:
        print('Verificare i nomi delle colonne')
else:
    print('tutto ok')

numero di colonne non corrispondente


In [11]:
cols_rename =  {
            'DENOMINAZIONE':    'denominazione',
            'c fiscale':        'cf', 
            'PARTITA IVA':      'piva',
            'N-REG-IMP':        'reg_imp_n', 
            'SEZ-REG-IMP':      'reg_imp_sez',
            'rea':              'rea',
            'N-ALBO-AA':        'n-albo_art', 
            
            'SIGLA PROVINCIA':  'prov',
            'COMUNE':           'comune',  
            'UL-SEDE':          'sede_ul',
            'n_sede':           'n_sede',

            'NG - Natura Giuridica':    'ng2', 
            'NG':                       'ng_esteso', 
            'tipo impresa':             'tipo_impresa',

            'DT-COST':                  'data_cost',
            'DT-ISCR-RI':               'data_isc_ri', 
            'DT-ISCR-RD':               'data_isc_rd', 
            'DT-ISCR-AA':               'data_isc_aa', 
            'DT-APER-UL':               'data_apert_ul', 
            'DT-INI-AT':                'data_ini_at',
            'DT-LIQUID':                'data_liquid',                        
            'DT-FINE-AA':               'data_fine_aa', 
            'dt cessazione attività':   'data_cess',

            'Cancellazione':    'cancellazione',   
            'Fallimento':       'fallimento',

            'AA-ADD':   'addetti_aaaa', 
            'IND':      'addetti_indip', 
            'DIP':      'addetti_dip',

             'CAPITALE':        'capitale',
             'VALUTA-CAPITALE': 'capitale_valuta',

             "ATTIVITA'":   'descrizione_attivita', 

            'stato impresa/ul':     'stato_impresa', 
            'Tipo Localizzazione':  'tipo_localizzazione',
            'tipo sede/ul1':'tipo_sedeul_1', 
            'tipo sede/ul2':'tipo_sedeul_2', 
            'tipo sede/ul3':'tipo_sedeul_3', 
            'tipo sede/ul4':'tipo_sedeul_4', 
            'tipo sede/ul5':'tipo_sedeul_5',
            "Presenza di sedi secondarie all'estero":                   'imp_sedi_ee',
            'Impresa estera con unità locale in Friuli Venezia Giulia': 'imp_eefvg',
            'Indicazione delle imprese che sono PMI':       'imp_pmi',
            'Indicazione delle imprese che sono start up':  'imp_startup', 
            'Impr Femminile':   'imp_femmilile',
            'Impr Giovane':     'imp_giovane', 
            'Impr Straniera':   'imp_straniera'
             }



In [12]:
#aggiorna i nomi delle colonne
df_anagrafica.columns = [cols_rename.get(x, x) for x in df_anagrafica.columns]

In [13]:
cols_to_drop = ['INDIRIZZO', 'STRAD', 'CAP',  'FRAZIONE', 
                'ALTRE-INDICAZIONI',  'pec','TELEFONO']
df_anagrafica.drop(columns = cols_to_drop, inplace = True)

In [14]:
#strip special characters
#chars_to_strip = '\\n\\t\\r|#'
cols_to_strip = [ 'denominazione',  'descrizione_attivita']
for ccc in cols_to_strip:
        if isinstance(ccc, str): 
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\n',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\t',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\r',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '"' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  "'" ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '|' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '_x000D_', ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '#' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.strip()
                print(ccc, " ok")

df_anagrafica['reg_imp_n'] = df_anagrafica['reg_imp_n'].str.replace(  '(', '', regex = True)
df_anagrafica['reg_imp_n'] = df_anagrafica['reg_imp_n'].str.replace(  ')', '', regex = True)

 

denominazione  ok
descrizione_attivita  ok


In [15]:
df_anagrafica

,id_localiz,denominazione,cf,piva,reg_imp_sez,reg_imp_n,rea,n-albo_art,prov,sede_ul,ng2,ng_esteso,tipo_impresa,data_cost,data_isc_ri,data_isc_rd,data_isc_aa,data_fine_aa,data_ini_at,data_apert_ul,data_liquid,cancellazione,data_cess,fallimento,addetti_aaaa,addetti_indip,addetti_dip,capitale,capitale_valuta,descrizione_attivita,stato_impresa,tipo_localizzazione,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovane,imp_straniera,comune,n_sede
0,1,PELLIZZARI SILVIO DI SEVERINO PELLIZZARI E C. ...,00000470310,00000470310,O,GO007-1352,37843,NaN,GO,SEDE,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4974-08-26 00:00:00,4996-02-19 00:00:00,4975-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,COMMERCIO AL MINUTO TAB. MERC. I - II (SOLTANT...,INATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,CORMONS - GO,0
1,2,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,SEDE,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,4969-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,TRIESTE - TS,0
2,3,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,UL-2,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5007-08-01 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,MONRUPINO - TS,2
3,4,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,UL-4,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5015-10-15 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,TRIESTE - TS,4
4,5,MOBILI BATTISTON & LOT - S.N.C. DI BATTISTON G...,00002130938,00002130938,O,PN033-2369,18374,NaN,PN,UL-1,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4973-10-09 00:00:00,4996-02-19 00:00:00,4973-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5008-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INATTIVA,UL - UNITÀ LOCALE,DEP - DEPOSITO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,PORDENONE - PN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127169,127170,ZOZZOLI ANTONIO,ZZZNTN78H14L195O,02564170302,A-P,UD-2009-7369,270774,UD-87280,UD,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,NaN,5009-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,"ATTIVITA : COSTRUZIONE, RISTRUTTURAZIONE DI ED...",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,PAULARO - UD,0
127170,127171,ESTETICA SAGITTARIO DI ZAZZERON ROBERTA,ZZZRRT70L67L424I,01139490328,A-P,TS-2008-2532,127748,TS-25173,TS,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5008-02-21 00:00:00,5008-02-19 00:00:00,5008-02-01 00:00:00,NaN,5008-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,ATTIVITA : ISTITUTO DI ESTETICA E VISAGISMO,ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,SI,NO,NO,TRIESTE - TS,0
127171,127172,ZAZZARINI ROBERTO,ZZZRRT74D06Z700T,01353640939,P,PN-1998-2076,70327,NaN,PN,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,4998-01-27 00:00:00,4998-01-15 00:00:00,NaN,NaN,4998-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,NaN,ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,SI,VALVASONE ARZENE - PN,0
127172,127173,ZOZZOLI SILVANA,ZZZSVN4

In [16]:
df_anagrafica['key_cfl'] = df_anagrafica['cf'] + '_'  + df_anagrafica['n_sede'].apply(str)
df_anagrafica

,id_localiz,denominazione,cf,piva,reg_imp_sez,reg_imp_n,rea,n-albo_art,prov,sede_ul,ng2,ng_esteso,tipo_impresa,data_cost,data_isc_ri,data_isc_rd,data_isc_aa,data_fine_aa,data_ini_at,data_apert_ul,data_liquid,cancellazione,data_cess,fallimento,addetti_aaaa,addetti_indip,addetti_dip,capitale,capitale_valuta,descrizione_attivita,stato_impresa,tipo_localizzazione,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovane,imp_straniera,comune,n_sede,key_cfl
0,1,PELLIZZARI SILVIO DI SEVERINO PELLIZZARI E C. ...,00000470310,00000470310,O,GO007-1352,37843,NaN,GO,SEDE,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4974-08-26 00:00:00,4996-02-19 00:00:00,4975-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,COMMERCIO AL MINUTO TAB. MERC. I - II (SOLTANT...,INATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,CORMONS - GO,0,00000470310_0
1,2,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,SEDE,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,4969-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,TRIESTE - TS,0,00002070324_0
2,3,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,UL-2,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5007-08-01 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,MONRUPINO - TS,2,00002070324_2
3,4,B.F.B. - CASA DI SPEDIZIONI S.R.L.,00002070324,00002070324,O,TS006-7084,65026,NaN,TS,UL-4,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,4969-01-30 00:00:00,4996-02-19 00:00:00,4969-01-30 00:00:00,NaN,NaN,NaN,5015-10-15 00:00:00,NaN,NaN,NaN,NaN,2015,2,13,20000,EURO,"SPEDIZIONI DOGANALI, DAL 30/01/1969",ATTIVA,UL - UNITÀ LOCALE,U - UFFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,TRIESTE - TS,4,00002070324_4
4,5,MOBILI BATTISTON & LOT - S.N.C. DI BATTISTON G...,00002130938,00002130938,O,PN033-2369,18374,NaN,PN,UL-1,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,4973-10-09 00:00:00,4996-02-19 00:00:00,4973-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5008-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INATTIVA,UL - UNITÀ LOCALE,DEP - DEPOSITO,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,PORDENONE - PN,1,00002130938_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127169,127170,ZOZZOLI ANTONIO,ZZZNTN78H14L195O,02564170302,A-P,UD-2009-7369,270774,UD-87280,UD,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,NaN,5009-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,"ATTIVITA : COSTRUZIONE, RISTRUTTURAZIONE DI ED...",ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,NO,PAULARO - UD,0,ZZZNTN78H14L195O_0
127170,127171,ESTETICA SAGITTARIO DI ZAZZERON ROBERTA,ZZZRRT70L67L424I,01139490328,A-P,TS-2008-2532,127748,TS-25173,TS,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,5008-02-21 00:00:00,5008-02-19 00:00:00,5008-02-01 00:00:00,NaN,5008-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,2012,1,NaN,NaN,NaN,ATTIVITA : ISTITUTO DI ESTETICA E VISAGISMO,ATTIVA,SE - SEDE PRINCIPALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,SI,NO,NO,TRIESTE - TS,0,ZZZRRT70L67L424I_0
127171,127172,ZAZZARINI ROBERTO,ZZZRRT74D06Z700T,01353640939,P,PN-1998-2076,70327,NaN,PN,SEDE,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,NaN,4998-01-27 00:00:00,4998-01-15 00:00:00,NaN,NaN,4998-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,1999,0,0,NaN,NaN,NaN,ATTIVA,SE - 

In [17]:
#write results to csv
file_risultati_anagrafica = data_dir + '\\' + 'imprese_anagrafica.csv'
df_anagrafica.to_csv(file_risultati_anagrafica, 
                    sep ='|',   
                    encoding='utf-8-sig', 
                    index=False)




### gestisco secondo foglio: codici


In [18]:
cols_source = [ 'c fiscale', 
                'SIGLA PROVINCIA', 
                'rea', 
                'loc', 
                'imp att', 
                'ateco 2007',
                'descrizione ateco 2007']

cols_rename =  {'c fiscale':                'cf', 
                'SIGLA PROVINCIA':          'prov', 
                'rea':                      'rea', 
                'loc':                      'loc_n', 
                'imp att':                  'ateco_tipo', 
                'ateco 2007':               'ateco',
                'descrizione ateco 2007':   'ateco_desc'
}

In [19]:
#carico i dati usando la terza riga per i nomi delle colonne (header = 2)
df_codici = xl.parse('FRIULI codice attività',  
                    header = 2,
                    dtype=str,
                    usecols = cols_source)[cols_source]
df_codici

,c fiscale,SIGLA PROVINCIA,rea,loc,imp att,ateco 2007,descrizione ateco 2007
0,00000470310,GO,37843,0,NaN,NaN,NaN
1,00002070324,TS,65026,0,I,52.29.1,Spedizionieri e agenzie di operazioni doganali
2,00002070324,TS,65026,0,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
3,00002070324,TS,65026,0,S,49.41,Trasporto di merci su strada
4,00002070324,TS,65026,2,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
...,...,...,...,...,...,...,...
268562,ZZZRRT70L67L424I,TS,127748,0,P,96.02.02,Servizi degli istituti di bellezza
268563,ZZZRRT74D06Z700T,PN,70327,0,P,01.11.4,"Coltivazioni miste di cereali, legumi da grane..."
268564,ZZZSVN46E60G381I,UD,227639,0,P,01.41,"Allevamento di bovini e bufale da latte, produ..."
268565,ZZZTMS74S21L483O,UD,235833,0,I,01.21,Coltivazione di uva


In [20]:
cols = df_codici.columns
if len(cols) != len(cols_source):
    print('numero di colonne non corrispondente')
    check_list = [item in cols  for item in cols_source]
    if False in check_list:
        print('Verificare i nomi delle colonne')
else:
    print('tutto ok')

tutto ok


In [21]:
df_codici.columns = [cols_rename.get(x, x) for x in df_codici.columns]

In [22]:
df_codici

,cf,prov,rea,loc_n,ateco_tipo,ateco,ateco_desc
0,00000470310,GO,37843,0,NaN,NaN,NaN
1,00002070324,TS,65026,0,I,52.29.1,Spedizionieri e agenzie di operazioni doganali
2,00002070324,TS,65026,0,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
3,00002070324,TS,65026,0,S,49.41,Trasporto di merci su strada
4,00002070324,TS,65026,2,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
...,...,...,...,...,...,...,...
268562,ZZZRRT70L67L424I,TS,127748,0,P,96.02.02,Servizi degli istituti di bellezza
268563,ZZZRRT74D06Z700T,PN,70327,0,P,01.11.4,"Coltivazioni miste di cereali, legumi da grane..."
268564,ZZZSVN46E60G381I,UD,227639,0,P,01.41,"Allevamento di bovini e bufale da latte, produ..."
268565,ZZZTMS74S21L483O,UD,235833,0,I,01.21,Coltivazione di uva


In [23]:
df_codici.reset_index(inplace = True)
df_codici

,index,cf,prov,rea,loc_n,ateco_tipo,ateco,ateco_desc
0,0,00000470310,GO,37843,0,NaN,NaN,NaN
1,1,00002070324,TS,65026,0,I,52.29.1,Spedizionieri e agenzie di operazioni doganali
2,2,00002070324,TS,65026,0,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
3,3,00002070324,TS,65026,0,S,49.41,Trasporto di merci su strada
4,4,00002070324,TS,65026,2,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
...,...,...,...,...,...,...,...,...
268562,268562,ZZZRRT70L67L424I,TS,127748,0,P,96.02.02,Servizi degli istituti di bellezza
268563,268563,ZZZRRT74D06Z700T,PN,70327,0,P,01.11.4,"Coltivazioni miste di cereali, legumi da grane..."
268564,268564,ZZZSVN46E60G381I,UD,227639,0,P,01.41,"Allevamento di bovini e bufale da latte, produ..."
268565,268565,ZZZTMS74S21L483O,UD,235833,0,I,01.21,Coltivazione di uva


In [24]:
#associo l'indice id_localiz alla tabella codici
df_temp = df_anagrafica [['cf','id_localiz','sede_ul', 'key_cfl']]
df_temp


,cf,id_localiz,sede_ul,key_cfl
0,00000470310,1,SEDE,00000470310_0
1,00002070324,2,SEDE,00002070324_0
2,00002070324,3,UL-2,00002070324_2
3,00002070324,4,UL-4,00002070324_4
4,00002130938,5,UL-1,00002130938_1
...,...,...,...,...
127169,ZZZNTN78H14L195O,127170,SEDE,ZZZNTN78H14L195O_0
127170,ZZZRRT70L67L424I,127171,SEDE,ZZZRRT70L67L424I_0
127171,ZZZRRT74D06Z700T,127172,SEDE,ZZZRRT74D06Z700T_0
127172,ZZZSVN46E60G381I,127173,SEDE,ZZZSVN46E60G381I_0


In [25]:
df_codici['key_cfl'] = df_codici['cf'] + '_'  + df_codici['loc_n']
df_codici

,index,cf,prov,rea,loc_n,ateco_tipo,ateco,ateco_desc,key_cfl
0,0,00000470310,GO,37843,0,NaN,NaN,NaN,00000470310_0
1,1,00002070324,TS,65026,0,I,52.29.1,Spedizionieri e agenzie di operazioni doganali,00002070324_0
2,2,00002070324,TS,65026,0,P,52.29.1,Spedizionieri e agenzie di operazioni doganali,00002070324_0
3,3,00002070324,TS,65026,0,S,49.41,Trasporto di merci su strada,00002070324_0
4,4,00002070324,TS,65026,2,P,52.29.1,Spedizionieri e agenzie di operazioni doganali,00002070324_2
...,...,...,...,...,...,...,...,...,...
268562,268562,ZZZRRT70L67L424I,TS,127748,0,P,96.02.02,Servizi degli istituti di bellezza,ZZZRRT70L67L424I_0
268563,268563,ZZZRRT74D06Z700T,PN,70327,0,P,01.11.4,"Coltivazioni miste di cereali, legumi da grane...",ZZZRRT74D06Z700T_0
268564,268564,ZZZSVN46E60G381I,UD,227639,0,P,01.41,"Allevamento di bovini e bufale da latte, produ...",ZZZSVN46E60G381I_0
268565,268565,ZZZTMS74S21L483O,UD,235833,0,I,01.21,Coltivazione di uva,ZZZTMS74S21L483O_0


In [26]:
# colelgamento tra tabella anagrafica e tabella codici
# nb è corretto chiamare le righe di anagrafica "localizzazioni"?
# nb non stiamo usando loc_n quindi il risultato è sbagliato!
# dal campi UL_SEDE bisogna estrarre il numero loc_n e connettere CF_loc

# cf_loc è il vero campo univoco

df_codici = pd.merge(df_temp, df_codici, left_on=['key_cfl'], right_on=['key_cfl'])


In [27]:
df_codici 

,cf_x,id_localiz,sede_ul,key_cfl,index,cf_y,prov,rea,loc_n,ateco_tipo,ateco,ateco_desc
0,00000470310,1,SEDE,00000470310_0,0,00000470310,GO,37843,0,NaN,NaN,NaN
1,00002070324,2,SEDE,00002070324_0,1,00002070324,TS,65026,0,I,52.29.1,Spedizionieri e agenzie di operazioni doganali
2,00002070324,2,SEDE,00002070324_0,2,00002070324,TS,65026,0,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
3,00002070324,2,SEDE,00002070324_0,3,00002070324,TS,65026,0,S,49.41,Trasporto di merci su strada
4,00002070324,3,UL-2,00002070324_2,4,00002070324,TS,65026,2,P,52.29.1,Spedizionieri e agenzie di operazioni doganali
...,...,...,...,...,...,...,...,...,...,...,...,...
269024,ZZZRRT70L67L424I,127171,SEDE,ZZZRRT70L67L424I_0,268562,ZZZRRT70L67L424I,TS,127748,0,P,96.02.02,Servizi degli istituti di bellezza
269025,ZZZRRT74D06Z700T,127172,SEDE,ZZZRRT74D06Z700T_0,268563,ZZZRRT74D06Z700T,PN,70327,0,P,01.11.4,"Coltivazioni miste di cereali, legumi da grane..."
269026,ZZZSVN46E60G381I,127173,SEDE,ZZZSVN46E60G381I_0,268564,ZZZSVN46E60G381I,UD,227639,0,P,01.41,"Allevamento di bovini e bufale da latte, produ..."
269027,ZZZTMS74S21L483O,127174,SEDE,ZZZTMS74S21L483O_0,268565,ZZZTMS74S21L483O,UD,235833,0,I,01.21,Coltivazione di uva


In [29]:
file_risultati_codici = data_dir + '\\' + 'imprese_codici.csv'
df_codici.to_csv(file_risultati_codici, sep ='|',   encoding='utf-8-sig', index=False)

In [30]:
print('end')


end
